In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nbdev import *
%nbdev_default_export base
%nbdev_default_class_level 3

Cells will be exported to geomechy.base,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_export
import pandas as pd

# Base
> all the base classes

## Base Dict

In [ ]:
%nbdev_export
class ItemDict(dict):
    
    def add(self, Id, item):
        self[Id] = item
        
    def get(self, Ids):
        return (self[Ids] if isinstance(Ids,int) else [self[Id] for Id in Ids] 
                if isinstance(Ids,list) else "NAN")

    def getId(self, Ids):
        return (list(self.keys())[Ids] if isinstance(Ids,int) else [list(self.keys())[Id] for Id in Ids] 
                if isinstance(Ids,list) else "NaN" )

## Base Properties

In [ ]:
%nbdev_export
class Properties:
    
    def __init__ (self, props={}):

        for key in props.keys():
            setattr(self, key, props[key])
    
    def __str__ (self):
        att_prop  = ''
        
        for att in dir(self):
            if att.startswith('__') or att.startswith('store'):
                continue
                
            att_prop += 'Attribute: ' + att + '\n'
            att_prop += str(getattr(self,att)) + '\n'

        return att_prop
    
    def __iter__ ( self ):
        propsList = []
        
        for att in dir(self):
            if att.startswith('__') or att.startswith('store'):
                continue
            propsList.append((att, getattr(self,att)))

        return iter(propsList)
    
    def store (self, key, val):
        setattr(self, key, val)

## Base Element

In [ ]:
%nbdev_export
class BaseElement(list):
    
    dofTypes = []
    
    def __init__ (self, nodes, props):
        list.__init__(self, nodes)
        self.history = {}
        self.current = {}

        for name,val in props:
            if name is "material":
                self.matProps = val
                self.mat = MaterialManager(self.matProps)
            else:
                setattr(self, name, val)
                
    def dofCount(self):
        return len(self)*len(self.dofTypes)
    
    def getNodes(self):
        return self
    
    def getType (self):
        return self.elemType
    
    def setHistoryParameter(self, name, val):
        self.current[name] = val
        
    def getHistoryParameter(self, name):
        return self.history[name]
    
    def commitHistory(self):
        self.history = self.current.copy()
        self.current = {}
        
        if hasattr(self, "mat"):
            self.mat.commitHistory()

## Base Constitutive

In [ ]:
%nbdev_export
class BaseConstitutive:
    
    def __init__(self, props):
        for name,val in props.items():
            setattr(self, name, val)

        self.history = pd.DataFrame({"i":[0], "d_epsilon":[0]})
        self.current = {"i":0, "d_epsilon":0}
        self.iIter   = -1

    def setIter(self, iIter):
        self.iIter = iIter

    def setHistoryParameter(self, name, val):
        if self.iIter == -1:
            self.history[name]=val
            self.current[name]=val
            return

    def getHistoryParameter(self, name, k):
        return self.history.loc[k,name]
    
    def setCurrentParameter(self, name, val):
        self.current[name]=val
        
    def getCurrentParameter(self, name):
        return self.current[name]
    
    def commitHistory(self):
        self.setIter(0)
        self.history = self.history.append(self.current, ignore_index=True)
        for key, val in self.current.items():
            self.current[key] = 0.
        

In [ ]:
elastic = BaseConstitutive({"E":500, "nu":0.3})
elastic.setHistoryParameter("p",0)
elastic.getCurrentParameter("p")
elastic.commitHistory()
elastic.history

,k,d_epsilon,p
0,0,0,0
1,0,0,0


In [ ]:
%nbdev_hide
notebook2script()

Converted 00_core.ipynb.
Converted 01_physics.ipynb.
Converted 02_constitutive.ipynb.
Converted 03_shape.ipynb.
Converted 04_io.ipynb.
Converted 06_materials.ipynb.
Converted 08_Modules.ipynb.
Converted 09_base.ipynb.
Converted 10_utils.ipynb.
Converted index.ipynb.
Converted test.ipynb.
